In [1]:
import numpy as np
from sklearn.datasets import make_moons
from sklearn.ensemble import AdaBoostClassifier
import plotly.plotly     as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [2]:
X, y = make_moons(n_samples=300, noise=0.2)

In [3]:
adaboost = AdaBoostClassifier(n_estimators=100)
adaboost.fit(X, y)

estimators = adaboost.estimators_

staged_classification = np.array(list(adaboost.staged_predict(X)))
staged_missclassified = staged_classification != y
staged_weights = np.ones(shape=staged_missclassified.shape)*5
staged_weights[staged_missclassified] = 15

In [4]:
h = .02
xrange = np.max(X[:, 0]) - np.min(X[:, 0])
yrange = np.max(X[:, 1]) - np.min(X[:, 1])
xs = np.arange(np.min(X[:, 0])-xrange*0.1, np.max(X[:, 0])+xrange*0.1, h)
ys = np.arange(np.min(X[:, 1])-xrange*0.1, np.max(X[:, 1])+xrange*0.1, h)
xx, yy = np.meshgrid(xs, ys)

In [5]:
staged_zz = np.array(list(adaboost.staged_predict(np.c_[xx.ravel(), yy.ravel()])))
staged_zz = staged_zz.reshape(len(staged_zz), xx.shape[0], xx.shape[1])

In [6]:
istage = 6

data   = [go.Scatter(x=X[:, 0], y=X[:, 1], mode='markers', 
                     marker=dict(color=y, colorscale='RdBu', size=staged_weights[istage])),
          go.Heatmap(x=xs, y=ys, z=staged_zz[istage], 
                     colorscale='RdBu', opacity=0.3, showscale=False)]
layout = go.Layout(autosize=False, width=500, height=500)
fig = go.Figure(data=data, layout=layout)

iplot(fig)